In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Урок 13. Модель BERT и GPT

Взять датасет
https://huggingface.co/datasets/merionum/ru_paraphraser
решить задачу парафраза

In [3]:
import pandas as pd
import numpy as np
import torch

from sklearn import preprocessing
from sklearn.metrics import f1_score


from transformers import AutoModelForSequenceClassification, BertTokenizer

In [4]:
from datasets import load_dataset

corpus_train = load_dataset('merionum/ru_paraphraser', data_files='train.jsonl')

Using custom data configuration merionum--ru_paraphraser-4a63244a00d42f51
Reusing dataset json (/root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-4a63244a00d42f51/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
df = pd.DataFrame(corpus_train['train'])[:200]
df.head()

,id,id_1,id_2,text_1,text_2,class
0,1,201,8159,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0
1,2,202,8158,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0
2,3,273,8167,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0
3,4,220,8160,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1
4,5,223,8160,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0


In [6]:
labels = df['class'].values

le = preprocessing.LabelEncoder()
le.fit(labels)

labels = le.transform(labels) 

In [7]:
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'

model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer = BertTokenizer.from_pretrained(model_name)

In [8]:
paraphrase = tokenizer(list(df['text_1']), list(df['text_2']), padding=True, return_tensors='pt')

In [9]:
paraphrase_classification_logits = model(**paraphrase).logits

In [10]:
paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1)

In [11]:
predictions = torch.argmax(paraphrase_results,dim=1)

In [12]:
f1_score(labels, predictions, average='macro')

0.4655555555555555